# Find proteins outside the 8q loss event (trans) that are commonly different between samples with and without the event

# Only looking at interacting proteins

## Setup

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
CHROMOSOME = "8"
ARM = "q"
TRANS_OR_CIS = "trans"

ttest_results_file = f"{CHROMOSOME}{ARM}_{TRANS_OR_CIS}effects_filtered.tsv"

ttest_results = pd.\
read_csv(ttest_results_file, sep="\t").\
rename(columns={"Name": "protein"}).\
set_index("protein")

In [3]:
ttest_results

,lscc_Database_ID,luad_Database_ID,brca_Database_ID,ovarian_Database_ID,brca_pvalue,colon_pvalue,hnscc_pvalue,lscc_pvalue,luad_pvalue,ovarian_pvalue,brca_diff,colon_diff,hnscc_diff,lscc_diff,luad_diff,ovarian_diff
protein,,,,,,,,,,,,,,,,
AAR2,NP_001258803.1,NP_001258803.1,NP_001258803.1,NP_001258803,0.829683,0.388351,0.954917,0.809374,0.858551,0.996388,0.167240,0.119473,-0.014591,-0.148427,0.062387,0.039881
ABCF2,NP_005683.2|NP_009120.1,NP_005683.2|NP_009120.1,NP_005683.2|NP_009120.1,NP_005683,0.916163,0.617549,0.952474,0.976446,0.371743,0.838909,-0.073970,0.070105,0.015746,0.031072,0.260041,0.129156
ABL1,NP_009297.2|NP_005148.2,NaN,NP_009297.2|NP_005148.2,NP_005148,0.857726,0.664714,0.404409,0.929574,NaN,0.996388,-0.148490,-0.095419,0.093299,0.044903,NaN,0.051351
ABLIM1,NP_001309811.1|NP_001309813.1|NP_001339369.1|N...,NP_001309811.1|NP_001309813.1|NP_001339369.1|N...,NP_001309811.1|NP_001309815.1,NP_001309811,0.946484,0.742462,0.675888,0.808364,0.529265,0.996388,-0.106391,-0.068128,-0.108047,0.277711,-0.344168,-0.107142
ABLIM1,NP_001309811.1|NP_001309813.1|NP_001339369.1|N...,NP_001309811.1|NP_001309813.1|NP_001339369.1|N...,NP_001309811.1|NP_001309815.1,NP_001003407,0.946484,0.742462,0.675888,0.808364,0.529265,0.996388,-0.106391,-0.068128,-0.108047,0.277711,-0.344168,-0.101455
ABLIM1,NP_001003407.1|NP_002304.3|NP_001309812.1|NP_0...,NP_001309811.1|NP_001309813.1|NP_001339369.1|N...,NP_001309811.1|NP_001309815.1,NP_001309811,0.946484,0.742462,0.675888,0.809374,0.529265,0.996388,-0.106391,-0.068128,-0.108047,0.347045,-0.344168,-0.107142
ABLIM1,NP_001003407.1|NP_002304.3|NP_001309812.1|NP_0...,NP_001309811.1|NP_001309813.1|NP_001339369.1|N...,NP_001309811.1|NP_001309815.1,NP_001003407,0.946484,0.742462,0.675888,0.809374,0.529265,0.996388,-0.106391,-0.068128,-0.108047,0.347045,-0.344168,-0.101455
ACACA,NP_942133.1|NP_942131.1|NP_942134.1|NP_942135.1,NP_942133.1|NP_942131.1|NP_942134.1|NP_942135.1,NP_942133.1|NP_942134.1|NP_942135.1,NP_942131,0.991357,0.567233,0.536248,0.982774,0.698895,0.996388,-0.008074,0.090998,0.112087,0.017680,0.220999,0.050508
ACTL6A,NP_004292.1|NP_817126.1|NP_057272.1,NaN,NP_004292.1|NP_829888.1,NP_004292,0.806093,0.388351,0.454635,0.809374,NaN,0.996388,0.187580,0.078619,0.124885,-0.201978,NaN,0.040739


## Reshape the input dataframe
We want to get our table to have these columns:
- cancer_type
- protein
- Database_ID
- change
- p_value

Since some cancer types have database IDs and some don't, we'll slice out and reshape the info for each cancer type individually.

In [4]:
cancer_types = sorted(ttest_results.columns.to_series().str.split("_", n=1, expand=True)[0].unique())

long_results = pd.DataFrame()

for cancer_type in cancer_types:
    cancer_df = ttest_results.\
    loc[:, ttest_results.columns.str.startswith(cancer_type)].\
    dropna(axis="index", how="all").\
    reset_index(drop=False)
    
    # If the cancer type has database IDs, make a separate column that has them.
    # If not, create a column of NaNs (so that the tables all match)
    if f"{cancer_type}_Database_ID" in cancer_df.columns:
        cancer_df = cancer_df.rename(columns={f"{cancer_type}_Database_ID": "Database_ID"})
    else:
        cancer_df = cancer_df.assign(Database_ID=np.nan)
        
    # Rename the pvalue and diff columns to not have the cancer type
    cancer_df = cancer_df.rename(columns={
        f"{cancer_type}_pvalue": "adj_p",
        f"{cancer_type}_diff": "change"
    }).\
    assign(cancer_type=cancer_type)
    
    # Reorder the columns
    cancer_df = cancer_df[["cancer_type", "protein", "Database_ID", "adj_p", "change"]]
    
    # Append to the overall dataframe
    long_results = long_results.append(cancer_df)

# Drop duplicate rows and reset the index
long_results = long_results[~long_results.duplicated(keep=False)].\
reset_index(drop=True)

In [5]:
long_results

,cancer_type,protein,Database_ID,adj_p,change
0,brca,AAR2,NP_001258803.1,0.829683,0.167240
1,brca,ABCF2,NP_005683.2|NP_009120.1,0.916163,-0.073970
2,brca,ABL1,NP_009297.2|NP_005148.2,0.857726,-0.148490
3,brca,ACACA,NP_942133.1|NP_942134.1|NP_942135.1,0.991357,-0.008074
4,brca,ACTL6A,NP_004292.1|NP_829888.1,0.806093,0.187580
5,brca,ADSL,NP_000017.1|NP_001304852.1|NP_001116850.1,0.985757,-0.026080
6,brca,AGR2,NP_006399.1,0.932960,0.244978
7,brca,AKAP13,NP_009131.2|NP_006729.4|NP_001257475.1,0.913668,0.070032
8,brca,AKAP5,NP_004848.3,0.900045,-0.246105
9,brca,AKT1,NP_005154.2,0.978505,0.035599


## Select the proteins with a significant change

In [6]:
prots = long_results[long_results["adj_p"] <= 0.05].reset_index(drop=True)

In [7]:
prots.groupby("cancer_type").count()[["protein"]]

,protein
cancer_type,
brca,1
colon,5
hnscc,4
lscc,1
luad,8


## Find how many cancers each protein was different in

In [8]:
prots_summary = prots.groupby("protein").agg(**{
    "cancers": ("cancer_type", lambda x: x.sort_values().drop_duplicates(keep="first").tolist())
})

prots_summary = prots_summary.\
assign(
    num_cancers=prots_summary["cancers"].apply(len),
    tmp_sort=prots_summary["cancers"].apply(lambda x: "".join(x))
).\
sort_values(by=["num_cancers", "tmp_sort"], ascending=[False, True]).\
drop(columns="tmp_sort")

prots_summary

,cancers,num_cancers
protein,,
PCM1,"[brca, colon, hnscc, lscc, luad]",5
ALDH1B1,[colon],1
LAMP1,[colon],1
SDF4,[colon],1
UBA5,[colon],1
DEF6,[hnscc],1
GNG2,[hnscc],1
HSP90AA1,[hnscc],1
CBL,[luad],1


In [9]:
prots_summary["num_cancers"].value_counts().sort_index(ascending=False)

5     1
1    14
Name: num_cancers, dtype: int64

## Save results

In [10]:
prots_summary = prots_summary.assign(
    cancers=prots_summary["cancers"].apply(lambda x: "_".join(x))
)

output_file = f"pancancer_summary_{CHROMOSOME}{ARM}_{TRANS_OR_CIS}_filtered.tsv"
prots_summary.to_csv(output_file, sep="\t")